## Load Prepared Dataset `synthetic-original.csv`

In [1]:
import pandas as pd
synthetic_original_df = pd.read_csv("../datasets/synthetic-original.csv")

## Applying the Augmentation Strategy to `synthetic-original.csv`

In [7]:
# test
synthetic_original_df = synthetic_original_df[0:200]

In [8]:
synthetic_original_df

,UserID,Docs,Action,Summaries
0,U335175,"[N70300, N66205, N68668, N113976, N61932, N724...","[click, skip, skip, skip, skip, skip, gen_summ...",13
1,U146053,"[N37826, N117079, N55055, N24616, N74267, N555...","[click, skip, click, click, skip, click, click...",12
2,U158889,"[N14064, N43380, N102856, N29737, N105956, N96...","[click, click, click, click, click, click, cli...",3
3,U22232,"[N80830, N91571, N122527, S7, N70758, N85067, ...","[click, skip, gen_summ, summ_gen, click, skip,...",6
4,U32515,"['N55509', 'N111634', 'N15992', 'N78883', 'N27...","['click', 'skip', 'click', 'click', 'click', '...",1
...,...,...,...,...
195,U217702,"['N118459', 'N94166', 'N50928', 'N66607', 'N88...","['click', 'click', 'click', 'click', 'skip', '...",5
196,U314295,"['N61448', 'N114190', 'N91695', 'N90094', 'N52...","['click', 'click', 'skip', 'skip', 'click', 'c...",4
197,U361310,"['N14872', 'N83830', 'N50693', 'N28859', 'N780...","['skip', 'skip', 'skip', 'click', 'click', 'cl...",0
198,U197363,"['N94432', 'N58081', 'N112057', 'N37762', 'S37...","['click', 'click', 'click', 'gen_summ', 'summ_...",4


In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast

synthetic_init_df = synthetic_original_df
num_users = 5
num_users_var = 25

gap = 5
gap_var = 5

impute_gap = 5
impute_gap_var = 5

length = 100
length_var = 200

iterations = 3
def safe_eval(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val

for _ in range(iterations):
    for i, row in tqdm(synthetic_init_df.iterrows(), total=synthetic_init_df.shape[0]):
        sample_df = synthetic_init_df.sample(n = num_users + np.random.randint(0, num_users_var + 1))
        
        for si, srow in sample_df.iterrows():
            gap = gap + np.random.randint(0, gap_var + 1)
            length = length + np.random.randint(0, length_var + 1)
            impute_seq_docs = safe_eval(srow['Docs'])[gap:gap + length]
            impute_seq_acts = safe_eval(srow['Action'])[gap:gap + length]
            
            if 'summ_gen' in impute_seq_acts:
                impute_gap = impute_gap + np.random.randint(0, impute_gap_var + 1)
                replace_seq_docs = safe_eval(row['Docs'])[:impute_gap] + impute_seq_docs + safe_eval(row['Docs'])[impute_gap:]
                replace_seq_acts = safe_eval(row['Action'])[:impute_gap] + impute_seq_acts + safe_eval(row['Action'])[impute_gap:]
                synthetic_init_df.at[i, 'Docs'] = replace_seq_docs
                synthetic_init_df.at[i, 'Action'] = replace_seq_acts
                assert len(synthetic_init_df.at[i, 'Docs']) == len(synthetic_init_df.at[i, 'Action'])
                synthetic_init_df.at[i, 'Summaries'] = replace_seq_acts.count('summ_gen')

# synthetic_init_df.to_csv('./datasets/synthetic-preprocessed.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 40.93it/s]


In [10]:
synthetic_init_df

,UserID,Docs,Action,Summaries
0,U335175,"[S2, N122297, N90902, N103034, N32119, N39558,...","[summ_gen, click, skip, click, click, click, s...",16
1,U146053,"[N62341, N26218, N118909, N29409, N30532, N781...","[click, skip, click, click, click, click, clic...",13
2,U158889,"[S125, N50846, N102360, N78899, N28779, N49411...","[summ_gen, click, click, click, click, skip, s...",4
3,U22232,"[N30442, N71915, N70388, N20440, N24548, N1212...","[click, click, click, click, click, gen_summ, ...",10
4,U32515,"['N55509', 'N111634', 'N15992', 'N78883', 'N27...","['click', 'skip', 'click', 'click', 'click', '...",1
...,...,...,...,...
195,U217702,"['N118459', 'N94166', 'N50928', 'N66607', 'N88...","['click', 'click', 'click', 'click', 'skip', '...",5
196,U314295,"['N61448', 'N114190', 'N91695', 'N90094', 'N52...","['click', 'click', 'skip', 'skip', 'click', 'c...",4
197,U361310,"['N14872', 'N83830', 'N50693', 'N28859', 'N780...","['skip', 'skip', 'skip', 'click', 'click', 'cl...",0
198,U197363,"['N94432', 'N58081', 'N112057', 'N37762', 'S37...","['click', 'click', 'click', 'gen_summ', 'summ_...",4


In [6]:
synthetic_original_df.iloc[3]['Action'].count('summ_gen')

7

In [7]:
python process_dataset.py path/to/input.csv path/to/output.csv --num-users 5 --num-users-var 25 --gap 2 --gap-var 5 --impute-gap 2 --impute-gap-var 5 --length 100 --length-var 200 --iterations 3


SyntaxError: invalid syntax (218130742.py, line 1)

In [8]:
!python "../src/data_manager.py"  augment-synthetic-data --help

                                                                               
 Usage: data_manager.py augment-synthetic-data [OPTIONS] INPUT_CSV OUTPUT_CSV  
                                                                               
 Process a synthetic dataset by modifying and updating rows based on given     
 parameters.                                                                   
 Args:     input_csv (str): Path to the input CSV file.     output_csv (str):  
 Path to save the processed CSV file.     num_users (int, optional): Base      
 number of users to sample. Defaults to 5.     num_users_var (int, optional):  
 Variability in the number of users to sample. Defaults to 25.     gap (int,   
 optional): Initial gap value. Defaults to 2.     gap_var (int, optional):     
 Variability in the gap value. Defaults to 5.     impute_gap (int, optional):  
 Initial impute gap value. Defaults to 2.     impute_gap_var (int, optional):  
 Variability in the impute gap value. De

In [1]:
!python "../src/data_manager.py" augment-synthetic-data  "../datasets/synthetic-original.csv" "../datasets/synthetic-original-augmented.csv" --num-users 5 --num-users-var 5 --gap 2 --gap-var 5 --impute-gap 2 --impute-gap-var 5 --length 25 --length-var 100 --iterations 1

^C


In [13]:
l = [1, 2,2,2,2,2,2,22,2,2,2, 5]
i = [3,3,3,3,3]

c = l[:5] + i + l[5:]

In [14]:
c

[1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 22, 2, 2, 2, 5]